# How to debug code

- corner cases: n=1 or maximum possible number
- think what the answer should be with pen and paper
- to measure CPU time since the start of the program, use "time.clock()"
- measure time for small, medium, and large tests

## Fabonacci numbers

- $F_{n} = 0$ if $n=0$
- $F_{n} = 1$ if $n=1$
- $F_{n} = F_{n-1} + F_{n-2}$ if $n>0$

### Naive algorithm

```
FibRecurse(n)
    if n <= 1
        return 1
    else
        FibRecurse(n-1) + FibRecurse(n-2)
```

### Efficient algorithm

```
FibList(n)
    create an array F[0...n]
    F[0] = 1
    F[1] = 1
    for i from 2 to n
        F[i] = F[i-1] + F[i-2]
    return F[n]
```

## Greatest common divisors

- for integers $a,b$, find the largest integer $d$ that divides both $a,b$

### Naive algorithm

```
gcd(a,b)
    best = 0
    for d from 1 to a+b
        if d % a == 0 and d % b ==0
            best = d
    return best
```

### Efficient algorithm

- lemma: let $a^{'}$ be the remainder when $a$ is divided by $b$, then gcd$(a,b)$ = gcd$(a^{'},b)$ = gcd$(b, a^{'})$

```
EuclidGCD(a,b)
    if b == 0
        return a
    a' = the remainder when a is divided by b
    return EuclidGCD(b,a')
```

# Greedy algorithms

## Car refueling problem

```
MinRefills(x,n,L)
    numRefills = 0
    currentRefill = 0
    while currentRefill <= n:
        lastRefill = currentRefull
        while (currentRefill <= n and x[currentRefill+1] - x[lastRefill] <= L):
            currentRefill = currentRefill + 1
        if currentRefill == lastRefill:
            return IMPOSSIBLE
        if currentRefill <= n:
            numRefills = numRefills + 1
    return numRefills
```

- currentRefill changes from $0$ to $n+1$, one by one
- numRefill changes from $0$ to at most $n$, one by one
- thus runs in $O(n)$

## Grouping children problem

naive algorithm
```
MinGroups(C)
    m = len(C)
    for each partition into groups: (C = G1 Union G2 Union ... Union Gk)
        good = true
        for i from 1 to k:
            if max(Gi) - min(Gi) > 1:
                good = false
        if good:
            m = min(m, k)
    return m
```

lemma
- the number of operations in MinGroups$(C)$ is at least $2^{n}$ where $n$ is the number of children in $C$ (runs in $\Omega{(2^{n})}$)

proof
- consider just partitions in two groups $C = G_{1} \cup G_{2}$
- size of $C$ is $n$
- each item can be included or excluded from $G_{1}$
- there are $2^{n}$ different $G_{1}$
- thus, at least $2^{n}$ operations

efficient algorithm
- covering points by segments
    - input: a set of $n$ points $x_{1} \dots x_{n} \in R$
    - output: the minimum number of segments of unit length needed to cover all the points
- assume $x_{1} \dots x_{n}$ are sorted
```
PointsCoverSorted(X1,...,Xn)
    R = {}
    i = 1
    while i <= n:
        [l,r] = [Xi, Xi + 1]
        R = R Union {[l, r]}
        i = i + 1
        while i <= n and Xi <= r:
            i = i + 1
    return R
```

lemma
- the running time of PointsCoverSorted is $O(n)$

proof
- $i$ changes from $1$ to $n$
- for each $i$, at most $1$ new segment

## Fractional knapsack problem

- while knapsack is not full
- choose item $i$ with maximum $\dfrac{v_{i}}{w_{i}}$
- if item fits into knapsack, take all of it
- otherwise take so much as to fill the knapsack
- return total value and amounts taken

```
Knapsack(W,w1,v1,...,wn,vn)
    A = [0,...,0]
    V = 0
    repeat n times:
        if W = 0:
            return (V,A)
        select i with wi > 0 and max vi/wi
        a = min(wi, W)
        V = V + a * vi / wi
        wi = wi - a
        A[i] = A[i] + a
        W = W - a
    return (V,A)
```

lemma
- the running time of Knapsack is $O(n^{2})$

proof
- select best item on each step is $O(n)$
- main loop is executed $n$ times

improvement
- sort items by decreasing $\dfrac{v}{w}$

```
Knapsack(W,w1,v1,...,wn,vn)
    A = [0,...,0]
    V = 0
    for i from 1 to n:
        if W = 0:
            return (V,A)
        a = min(wi, W)
        V = V + a * vi / wi
        wi = wi - a
        A[i] = A[i] + a
        W = W - a
    return (V,A)
```
- now each iteration is $O(1)$
- Knapsack after sorting is $O(n)$
- sort + Knapsack is $O(nlogn)$

# Divide and conquer algorithms

```
LinearSearch(A, low, high, key) # recursive
    if high < low:
        return NOT_FOUND
    if A[low] = key:
        return low
    return LinearSearch(A, low+1, high, key)
```

recurrence
- $T(n) = T(n-1) + c$

```
BinarySearch(A, low, high, key) # sorted array
    if high < low:
        return low-1
    mid = math.floor(low + (high-low)/2)
    if key = A[mid]:
        return mid
    elif key < A[mid]:
        return BinarySearch(A, low, mid-1, key)
    else:
        return BinarySearch(A, mid+1, high, key)
```

recurrence
- $T(n) = T(\left\lfloor{\dfrac{n}{2}}\right\rfloor) + c$
- $T(0) = c$

```
Multpoly(A,B,n) # naive algorithm O(n^2)
    product = array[2n-1]
    for i from 0 to 2n-2:
        product[i] = 0
    for i from 0 to n-1:
        for j from 0 to n-1:
            product[i+j] = product[i+j] + A[i]*B[j]
    return product
```

```
Mult2(A,B,n,al,bl) # naive divide and conquer algorithm O(n^2)
    R = array[0...2n-1]
    if n=1:
        R[0] = A[al]*B[bl]
        return R
    R[0...n-2] = Mult2(A,B,n/2,al,bl)
    R[n...2n-2] = Mult2(A,B,n/2,al+n/2,bl+n/2)
    D0E1 = Mult2(A,B,n/2,al,bl+n/2)
    D1E0 = Mult2(A,B,n/2,al+n/2,bl)
    R[n/2...n+n/2-2] = D0E1 + D1E0
    return R
```

# Dynamic programming

What is the minimum number of coins needed to return the change (Greedy algorithm does not work for random denominators)

Recurrence
- minNumCoins(money) = min{minNumCoins(money-coin1)+1, minNumCoins(money-coin2)+1, minNumCoins(money-coin3)+1,...}

```
DPChange(money, coins)
MinNumCoins(0) = 0
for m from 1 to money:
    MinNumCoins(m) = 10000000000
    for i from 1 to |coins|:
        if m >= coin(i):
            NumCoins = MinNumCoins(m-coin(i))+1
            if NumCoins < MinNumCoins(m):
                MinNumCoins(m) = NumCoins
return MinNumCoins(money)
```

Given two string, mismatch penalty $\mu$ and indel penalty $\sigma$, find the alignment of strings that maximizes the score
- Above problem can be written as: find the minimum number of operations (insertion, deletion, substitution of symbols) to transform one string to another
    - the last column of an optimal alignment is one of insertion / deletion / mismatch / match
    - let $D(i,j)$ be the edit distance of an $i$-prefix $A[1 \dots i]$ and a $j$-prefix $B[1 \dots j]$
    - then $D(i,j) = min(D(i,j-1)+1, D(i-1,j)+1, D(i-1,j-1)+1$ if $A[i] \ne B[j], D(i-1,j-1)$ if $A[i] = B[j])$
    
```
EditDistance(A[1...n], B[1...m])
D(i,0) = i for all i
D(0,j) = j for all j
for j from 1 to m:
    for i from 1 to n:
        insertion = D(i, j-1)+1
        deletion = D(i-1, j)+1
        match = D(i-1, j-1)
        mismatch = D(i-1, j-1)+1
        if A[i] = B[j]:
            D(i,j) = min(insertion, deletion, match)
        else
            D(i,j) = min(insertion, deletion, mismatch)
return D(n,m)
```

```
OutputAlignment(i,j):
if i=0 and j=0:
    return
if i>0 and D(i,j) = D(i-1,j)+1:
    OutputAlignment(i-1,j)
    print(A[i])
    print("-")
elif j>0 and D(i,j) = D(i,j-1)+1:
    OutputAlignment(i,j-1)
    print("-")
    print(B[i])
else:
    OutputAlignment(i-1,j-1)
    print(A[i])
    print(B[i])
```

Knapsack
- input: weights $w_{1} \dots w_{n}$, values $v_{1} \dots v_{n}$, $n$ items, $W$ total weight
- output: max value of items whose value does not exceed $W$

Recurrence
- $i^{th}$ item is either used or not
    - $value(w,i)$ equals to: $max[value(w-w_{i}, i-1) + v_{i}, value(w, i-1)]$

```
Knapsack(w) with repetition
value(0) = 0
for w from 1 to W:
    value(w) = 0
    for i from 1 to n:
        if wi <= w:
            val = value(w-wi) + vi
            if val > value(w)
                value(w) = val
return value(W)
```

```
Knapsack(w) without repetition
initialize all values(0,j) = 0
initialize all values(w,0) = 0
for i from 1 to n:
    for w from 1 to W:
         value(w,i) = value(w,i-1)
         if wi <= w:
             val = value(w-wi, i-1) + vi
             if value(w,i) < val:
                 value(w,i) = val
return value(W,n)
```

```
Kanpsack(w) Memoization
if w is in hash-table
    return value(w)
value(w) = 0
for i from 1 to n:
    if wi <= w:
        val = Knapsack(w-wi) + vi
        if val > value(w)
            value(w) = val
insert value(w) into hash table with key w
return value(w)
```

Placing parentheses
- input: digits $d_{1} \dots d_{n}$ and ops $op_{1} \dots op_{n-1}$
- output: order of applying these operations that maximizes the value of the expression

Recurrence
- let $E$ be a subexpression
    - $d_{i}op_{i} \dots op_{j-1}d_{j}$
- subproblems
    - $M(i,j)$ = max value of $E_{i,j}$
    - $m(i,j)$ = min value of $E_{i,j}$
- $M(i,j) = max[M(i,j) op_{k} M(k+1,j), M(i,j) op_{k} m(k+1,j), m(i,j) op_{k} M(k+1,j), M(i,j) op_{k} m(k+1,j)]$
- $m(i,j) = min[M(i,j) op_{k} M(k+1,j), M(i,j) op_{k} m(k+1,j), m(i,j) op_{k} M(k+1,j), M(i,j) op_{k} m(k+1,j)]$

```
MinAndMax(i,j)
min = 100000000
max = -100000000
for k from i to j-1
    a = M(i,k) opk M(k+1,j)
    b = M(i,k) opk m(k+1,j)
    c = m(i,k) opk M(k+1,j)
    d = m(i,k) opk m(k+1,j)
    min = min(min, a, b, c, d)
    max = max(max, a, b, c, d)
return (min, max)
```

```
Parentheses(d1 op1 ... dn)
for i from 1 to n
    m(i,i) = di
    M(i,i) = di
for s from 1 to n-1
    for i from 1 to n-s
        j = i + s
        m(i,j), M(i,j) = MinAndMax(i,j)
return M(1,n)
```